In [2]:
import numpy as np 
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import open3d as o3d
from scipy.optimize import minimize
from scipy.linalg import lstsq

path_norm = "pclouds/boxunion_uniform100k_ddist_minmax_layers.normals"
path_pcd = "pclouds/boxunion_uniform100k_ddist_minmax_layers.xyz"

normals = np.loadtxt(path_norm)
points = np.loadtxt(path_pcd)
# print(normals.shape, pcd.shape)    

'''visualization'''

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.normals = o3d.utility.Vector3dVector(normals)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

'''neighborhood (normal init)'''

def find_neighs(points, k=250):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm= 'kd_tree').fit(points)
    distances, indices = nbrs.kneighbors(points) 
    return distances, indices

def pca_plane(points):
    # print(points.shape)
    pca = PCA(n_components=3)
    pca.fit(points)
    return pca.components_[2]/np.linalg.norm(pca.components_[2])

def scipy_ls(points):
    a = np.hstack((points[:, :2], np.ones(points.shape[0]).reshape(-1, 1)))
    b= points[:, 2]
    abc, residue, rank, s = lstsq(a, b) 
    return abc/np.linalg.norm(abc)

def plane_cost(params, points):
    a, b, c, d = params
    # Compute the signed distances from each point to the plane
    distances = np.abs(a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d) / np.sqrt(a**2 + b**2 + c**2)
    # Return the sum of squared distances as the cost
    return np.sum(distances**2)

def scipy_min(points):
    ini_guess= np.array([1.0,1.0,1.0,1.0])
    res = minimize(plane_cost, ini_guess, args=(points), method='Nelder-Mead', tol=1e-6)
    return res.x[:3]/np.linalg.norm(res.x[:3])

dists, indices = find_neighs(points, 250)
# print(pcd[indices].shape)
neighs = points[indices]

norm_esti_pca = Parallel(n_jobs=-1)(delayed(pca_plane)(point_neighs) for point_neighs in neighs)
norm_esti_ls = Parallel(n_jobs=-1)(delayed(scipy_ls)(point_neighs) for point_neighs in neighs)
norm_esti_min = Parallel(n_jobs=-1)(delayed(scipy_min)(point_neighs) for point_neighs in neighs)



In [3]:

norm_= normals/np.linalg.norm(normals, axis=1)[:,np.newaxis]
dot_pca = np.abs(np.diag(np.dot(norm_esti_pca[:300] , norm_[:300,:].T))).mean()
dot_ls = np.abs(np.diag(np.dot(norm_esti_ls[:300] , norm_[:300,:].T))).mean()
dot_min = np.abs(np.diag(np.dot(norm_esti_min[:300] , norm_[:300,:].T))).mean()

print(dot_pca, dot_ls, dot_min)

0.9615212762900998 0.6660288920804829 0.9491623252072406


In [8]:
type(norm_)

numpy.ndarray

In [4]:
import os

    # Get a list of all files in the folder
files = os.listdir("pclouds")

# Create a dictionary to store file lists based on extensions
file_dict = {}

# Iterate over each file and categorize based on extension
for file in files:
    filename, extension = os.path.splitext(file)
    print(filename, extension)
    break
    

armadillo100k_noise_brown_4.00e-02 .curv
Boxy_smooth100k_noise_white_1.00e-02 .pidx
armadillo100k_noise_white_8.00e-02 .xyz
sheet_analytic100k_noise_white_1.00e-02 .pidx
fandisk100k_noise_brown_5.00e-02 .xyz
sphere_analytic100k_noise_white_8.00e-02 .xyz
netsuke100k_noise_white_7.00e-02 .normals
galera100k_noise_white_1.00e-01 .curv
sphere100k_ddist_minmax_layers .curv
boxunion2100k_noise_white_5.00e-02 .normals
armadillo100k_noise_white_1.00e-02 .curv
icosahedron100k_ddist_minmax_layers .normals
boxunion2100k_noise_white_8.00e-02 .xyz
star_smooth100k_noise_white_7.00e-02 .curv
star_smooth100k_noise_white_1.00e-01 .pidx
pipe100k_ddist_minmax .xyz
Cup34100k_ddist_minmax_layers .normals
galera100k_ddist_minmax .pidx
cylinder_analytic100k_noise_white_3.00e-02 .normals
flower100k_noise_white_9.00e-02 .xyz
flower100k_noise_white_4.00e-02 .curv
icosahedron100k_noise_white_2.00e-02 .normals
tortuga100k_ddist_minmax_layers .pidx
boxunion2100k .curv
boxunion_uniform100k .curv
Liberty100k_noise_w